In [358]:
import pandas
import nltk
import repportClass as rc
import LinguisticClasses as lc
import IOfonctions as io

# Import Excel File

In [359]:
data = pandas.read_csv('dump.csv', delimiter=',', encoding='latin-1')
data

,incident_id,event_type,incident_description,descriptor,coordinator_comments,investigation_narrative,acute_medical_harm,dosimetric_impact,latent_medical_harm,functional_work_area,...,number_fractions_prescribed,number_fractions_incorrect,hardware_manufacturer_model,software_manufacturer_model,body_region_treated,treatment_intent,ameliorating_actions,safety_barriers_failed,safety_barriers_prevented,actions_reduce_risk
0,2511,Actual incident,Bolus was not prescribed by MD. Pt received 2 ...,Non prescribed bolus,NaN,NaN,None,Minor,No,TrueBeam 3,...,25.0,2.0,NaN,NaN,Thorax,NaN,Other,Verification of treatment accessories,NaN,NaN
1,2521,Actual incident,Noticed by nutistionist (pt had app. with supp...,skin breakdown post RT,NaN,I handed over the investigation of incident to...,Mild,None,Unknown,STx 1,...,25.0,0.0,NaN,NaN,Head and neck,Curative (Radical),NaN,NaN,NaN,NaN
2,2562,Actual incident,"Ready for treatment task sent,completed,howeve...",Call patient task never sent,NaN,NaN,None,None,Unknown,STx 2,...,10.0,0.0,NaN,NaN,Thorax,Curative (Radical),Staff debriefing or counselling,Time out/verbalization/call-back,NaN,Improved compliance with existing policies or ...
3,2560,Near miss,"Physics QA completed, but ready for treatment ...",ready for treatment tasking error,NaN,tasking error,NaN,NaN,NaN,Tasking,...,25.0,NaN,NaN,NaN,Thorax,Curative (Radical),Education or training,Verification of relevant clinical information,Verification of relevant clinical information,Process standardization
4,2526,Near miss,Typo in prescription document. It was prescrib...,prescription error,NaN,NaN,NaN,NaN,NaN,STx 2,...,5.0,NaN,NaN,NaN,NaN,NaN,Other,None,Independent confirmation of dose,NaN
5,2513,Actual incident,Started patient's treatments on a Friday inste...,New Start on Friday,NaN,NaN,None,None,Unknown,STx 1,...,20.0,2.0,NaN,NaN,Head and neck,Curative (Radical),Staff debriefing or counselling & Education or...,Verification of relevant clinical information ...,NaN,Reminder(s) or checklist(s) & Improved complia...
6,2570,Actual incident,Pt was treated on TB 6on Feb 1st due to Tb3 be...,Lost vacloc,NaN,NaN,None,Minor,No,TrueBeam 6,...,16.0,0.0,NaN,NaN,Thorax,Curative (Radical),Education or training,None,None,Leadership action - Culture change & Staff rem...
7,2518,Near miss,Bolus not prescibed,Bolus,NaN,NaN,NaN,NaN,NaN,TrueBeam 3,...,25.0,NaN,NaN,NaN,Thorax,Curative (Radical),Radiation treatment plan revision,Oncologist review of treatment plan,Verification of treatment accessories,Additional education or training
8,2543,Actual incident,"Patient was lost(forgotten) in system, plan re...",Lack of adequate planning time.,NaN,patient lost in process,None,None,No,Treatment Planning,...,NaN,NaN,NaN,NaN,NaN,NaN,Education or training,Other,NaN,Process standardization
9,2541,Actual incident,Oncologist complained why patient was not cont...,Tasking Error,NaN,tasking error,None,None,No,Treatment Planning,...,NaN,NaN,NaN,NaN,NaN,NaN,Education or training,Verification of relevant clinical information,Verification of relevant clinical information,Process standardization


# Create Near Miss Column

In [360]:
data['near_miss'] = data['event_type'].apply(lambda x: 1 if x=='Near miss' else 0)
data

,incident_id,event_type,incident_description,descriptor,coordinator_comments,investigation_narrative,acute_medical_harm,dosimetric_impact,latent_medical_harm,functional_work_area,...,number_fractions_incorrect,hardware_manufacturer_model,software_manufacturer_model,body_region_treated,treatment_intent,ameliorating_actions,safety_barriers_failed,safety_barriers_prevented,actions_reduce_risk,near_miss
0,2511,Actual incident,Bolus was not prescribed by MD. Pt received 2 ...,Non prescribed bolus,NaN,NaN,None,Minor,No,TrueBeam 3,...,2.0,NaN,NaN,Thorax,NaN,Other,Verification of treatment accessories,NaN,NaN,0
1,2521,Actual incident,Noticed by nutistionist (pt had app. with supp...,skin breakdown post RT,NaN,I handed over the investigation of incident to...,Mild,None,Unknown,STx 1,...,0.0,NaN,NaN,Head and neck,Curative (Radical),NaN,NaN,NaN,NaN,0
2,2562,Actual incident,"Ready for treatment task sent,completed,howeve...",Call patient task never sent,NaN,NaN,None,None,Unknown,STx 2,...,0.0,NaN,NaN,Thorax,Curative (Radical),Staff debriefing or counselling,Time out/verbalization/call-back,NaN,Improved compliance with existing policies or ...,0
3,2560,Near miss,"Physics QA completed, but ready for treatment ...",ready for treatment tasking error,NaN,tasking error,NaN,NaN,NaN,Tasking,...,NaN,NaN,NaN,Thorax,Curative (Radical),Education or training,Verification of relevant clinical information,Verification of relevant clinical information,Process standardization,1
4,2526,Near miss,Typo in prescription document. It was prescrib...,prescription error,NaN,NaN,NaN,NaN,NaN,STx 2,...,NaN,NaN,NaN,NaN,NaN,Other,None,Independent confirmation of dose,NaN,1
5,2513,Actual incident,Started patient's treatments on a Friday inste...,New Start on Friday,NaN,NaN,None,None,Unknown,STx 1,...,2.0,NaN,NaN,Head and neck,Curative (Radical),Staff debriefing or counselling & Education or...,Verification of relevant clinical information ...,NaN,Reminder(s) or checklist(s) & Improved complia...,0
6,2570,Actual incident,Pt was treated on TB 6on Feb 1st due to Tb3 be...,Lost vacloc,NaN,NaN,None,Minor,No,TrueBeam 6,...,0.0,NaN,NaN,Thorax,Curative (Radical),Education or training,None,None,Leadership action - Culture change & Staff rem...,0
7,2518,Near miss,Bolus not prescibed,Bolus,NaN,NaN,NaN,NaN,NaN,TrueBeam 3,...,NaN,NaN,NaN,Thorax,Curative (Radical),Radiation treatment plan revision,Oncologist review of treatment plan,Verification of treatment accessories,Additional education or training,1
8,2543,Actual incident,"Patient was lost(forgotten) in system, plan re...",Lack of adequate planning time.,NaN,patient lost in process,None,None,No,Treatment Planning,...,NaN,NaN,NaN,NaN,NaN,Education or training,Other,NaN,Process standardization,0
9,2541,Actual incident,Oncologist complained why patient was not cont...,Tasking Error,NaN,tasking error,None,None,No,Treatment Planning,...,NaN,NaN,NaN,NaN,NaN,Education or training,Verification of relevant clinical information,Verification of relevant clinical information,Process standardization,0


# Create Lists of NLTK Tags

In [361]:
tags = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB']
noun_tags = ['NN', 'NNS', 'NNP', 'NNPS']
verb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

# Word Count Method

In [362]:
def wc(st):
    return len(st.split(' '))
wc ('During chart QA, it was noticed that a 0.3cm')

9

# Tag Word by Category Method

In [363]:
def tag(st):
    return nltk.pos_tag(st.split())
tag('Missing info. on sim/ct sim sheet')

[('Missing', 'VBG'),
 ('info.', 'NN'),
 ('on', 'IN'),
 ('sim/ct', 'JJ'),
 ('sim', 'JJ'),
 ('sheet', 'NN')]

# Token Retrieval by NLTK Tags Method

In [400]:
def this_tag(st, tl):
    return [w for (w, t) in tag(st) if t in tl]
this_tag('Missing info. on sim/ct sim sheet', ['NN'])

['info.', 'sheet']

# Token Retrieval by cTakes Annotation Methods

In [365]:
def ctakes_verbs(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    return ob.getFrequencyPOS()[0]
ctakes_verbs(2526)

2

In [366]:
def ctakes_nouns(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    return ob.getFrequencyPOS()[1]
ctakes_nouns(2526)

6

In [367]:
def ctakes_prepositions(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    return ob.getFrequencyPOS()[2]
ctakes_prepositions(2526)

3

In [368]:
def ctakes_pronouns(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    return ob.getFrequencyPOS()[3]
ctakes_pronouns(2526)

1

In [369]:
def ctakes_adverbs(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    return ob.getFrequencyPOS()[4]
ctakes_adverbs(2526)

0

In [370]:
def ctakes_adjectives(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    return ob.getFrequencyPOS()[5]
ctakes_adjectives(2526)

0

In [371]:
def ctakes_other(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    return ob.getFrequencyPOS()[6]
ctakes_other(2526)

1

In [372]:
def ctakes_medical(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    mentions = []
    for it in ob.getMedicalTerms():
        if it.typeOfMedicalTerm==1:
            mentions.append(it.getText())
    return mentions
ctakes_medical(2511)

[]

In [373]:
def ctakes_procedure(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    mentions = []
    for it in ob.getMedicalTerms():
        if it.typeOfMedicalTerm==2:
            mentions.append(it.getText())
    return mentions
ctakes_procedure(2511)

['treatments']

In [374]:
def ctakes_symptom(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    mentions = []
    for it in ob.getMedicalTerms():
        if it.typeOfMedicalTerm==3:
            mentions.append(it.getText())
    return mentions
ctakes_symptom(2511)

[]

In [375]:
def ctakes_disease(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    mentions = []
    for it in ob.getMedicalTerms():
        if it.typeOfMedicalTerm==4:
            mentions.append(it.getText())
    return mentions
ctakes_disease(2511)

[]

In [376]:
def ctakes_anatomy(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    mentions = []
    for it in ob.getMedicalTerms():
        if it.typeOfMedicalTerm==5:
            mentions.append(it.getPreferedText())
    return mentions
ctakes_anatomy(2521)

['skin', 'skin']

In [377]:
def ctakes_all(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    mentions = []
    for it in ob.getMedicalTerms():
        mentions.append(it.getText())
    return mentions
ctakes_all(2521)

['support', 'treatment', 'treatments', 'rad', 'skin', 'skin']

In [378]:
def brunet(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    return ob.getBrunetIndex()
brunet(2521)

8.156964513413685

In [379]:
def ttr(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    return ob.getTTR()
ttr(2521)

0.7735849056603774

In [380]:
def hapex_legomena(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    return ob.getHapaxLegomena()
hapex_legomena(2521)

41

In [381]:
def hapex_dislegomena(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    return ob.getHapaxDislegomena()
hapex_dislegomena(2521)

10

In [401]:
def sichel(id):
    fi = '/Users/erickvelazquez/Documents/RealDataSet/SeparatedRecords/output_Incident_description/'+str(id)+'.txt.xmi'
    ob = rc.RecordCTakesProcessed(io.readXMIFile(fi), id)
    return ob.getSichel()
sichel(2521)

0.18867924528301888

# cTakes Analyses

In [390]:
def ctakes_analyze(df):
    df['incident_description_ctakes_verbs'] = df['incident_id'].apply(ctakes_verbs)
    df['incident_description_ctakes_nouns'] = df['incident_id'].apply(ctakes_nouns)
    df['incident_description_ctakes_prepositions'] = df['incident_id'].apply(ctakes_prepositions)
    df['incident_description_ctakes_pronouns'] = df['incident_id'].apply(ctakes_pronouns)
    df['incident_description_ctakes_adverbs'] = df['incident_id'].apply(ctakes_adverbs)
    df['incident_description_ctakes_adjectives'] = df['incident_id'].apply(ctakes_adjectives)
    df['incident_description_ctakes_other'] = df['incident_id'].apply(ctakes_other)
    df['incident_description_ctakes_medical'] = df['incident_id'].apply(lambda x: len(ctakes_medical(x)))
    df['incident_description_ctakes_procedure'] = df['incident_id'].apply(lambda x: len(ctakes_procedure(x)))
    df['incident_description_ctakes_symptom'] = df['incident_id'].apply(lambda x: len(ctakes_symptom(x)))
    df['incident_description_ctakes_disease'] = df['incident_id'].apply(lambda x: len(ctakes_disease(x)))
    df['incident_description_ctakes_anatomy'] = df['incident_id'].apply(lambda x: len(ctakes_anatomy(x)))
    df['incident_description_ctakes_all'] = df['incident_id'].apply(lambda x: len(ctakes_all(x)))
    df['incident_description_brunet'] = df['incident_id'].apply(brunet)
    df['incident_description_ttr'] = df['incident_id'].apply(ttr)
    df['incident_description_hapex_legomena'] = df['incident_id'].apply(hapex_legomena)
    df['incident_description_hapex_dislegomena'] = df['incident_id'].apply(hapex_dislegomena)
    df['incident_description_sichel'] = df['incident_id'].apply(sichel)
    return df

In [392]:
ctakes_analyzed = ctakes_analyze(data)
ctakes_analyzed

,incident_id,event_type,incident_description,descriptor,coordinator_comments,investigation_narrative,acute_medical_harm,dosimetric_impact,latent_medical_harm,functional_work_area,...,incident_description_ctakes_procedure,incident_description_ctakes_symptom,incident_description_ctakes_disease,incident_description_ctakes_anatomy,incident_description_ctakes_all,incident_description_brunet,incident_description_ttr,incident_description_hapex_legomena,incident_description_hapex_dislegomena,incident_description_sichel
0,2511,Actual incident,Bolus was not prescribed by MD. Pt received 2 ...,Non prescribed bolus,NaN,NaN,None,Minor,No,TrueBeam 3,...,1,0,0,0,1,5.120753,0.909091,10,1,0.090909
1,2521,Actual incident,Noticed by nutistionist (pt had app. with supp...,skin breakdown post RT,NaN,I handed over the investigation of incident to...,Mild,None,Unknown,STx 1,...,3,0,1,2,6,8.156965,0.773585,41,10,0.188679
2,2562,Actual incident,"Ready for treatment task sent,completed,howeve...",Call patient task never sent,NaN,NaN,None,None,Unknown,STx 2,...,1,0,0,0,1,5.509168,0.928571,13,1,0.071429
3,2560,Near miss,"Physics QA completed, but ready for treatment ...",ready for treatment tasking error,NaN,tasking error,NaN,NaN,NaN,Tasking,...,1,0,0,0,1,4.659549,1.000000,10,0,0.000000
4,2526,Near miss,Typo in prescription document. It was prescrib...,prescription error,NaN,NaN,NaN,NaN,NaN,STx 2,...,0,0,0,0,0,5.397375,0.818182,9,2,0.181818
5,2513,Actual incident,Started patient's treatments on a Friday inste...,New Start on Friday,NaN,NaN,None,None,Unknown,STx 1,...,1,0,0,0,1,6.253253,0.800000,12,2,0.133333
6,2570,Actual incident,Pt was treated on TB 6on Feb 1st due to Tb3 be...,Lost vacloc,NaN,NaN,None,Minor,No,TrueBeam 6,...,0,0,0,0,0,6.336893,0.894737,17,1,0.052632
7,2518,Near miss,Bolus not prescibed,Bolus,NaN,NaN,NaN,NaN,NaN,TrueBeam 3,...,0,0,0,0,0,2.475541,1.000000,3,0,0.000000
8,2543,Actual incident,"Patient was lost(forgotten) in system, plan re...",Lack of adequate planning time.,NaN,patient lost in process,None,None,No,Treatment Planning,...,0,1,1,0,2,5.274793,1.000000,14,0,0.000000
9,2541,Actual incident,Oncologist complained why patient was not cont...,Tasking Error,NaN,tasking error,None,None,No,Treatment Planning,...,0,0,0,0,0,5.720450,0.937500,15,1,0.062500


In [393]:
ctakes_analyzed.to_csv('ctakes_analyzed.csv')

# NLTK Analysis

In [395]:
def nltk_analyze(df, cls):
    for cl in cls:
        # NLTK Analyses
        df[cl+'nltk_wc'] = df[cl].apply(wc) # Word Count
        df[cl+'nltk_n'] = df[cl].apply(lambda x: len(this_tag(x, noun_tags))) # Noun Count
        df[cl+'nltk_v'] = df[cl].apply(lambda x: len(this_tag(x, verb_tags))) # Verb Count
        for tag in tags: # Counts of All 35 Tags
            df[cl+'_nltk_'+tag] = df[cl].apply(lambda st: len(this_tag(st, [tag])))
    return df

In [397]:
analyzed = nltk_analyze(ctakes_analyzed, ['incident_description', 'descriptor'])
analyzed

,incident_id,event_type,incident_description,descriptor,coordinator_comments,investigation_narrative,acute_medical_harm,dosimetric_impact,latent_medical_harm,functional_work_area,...,descriptor_nltk_VB,descriptor_nltk_VBD,descriptor_nltk_VBG,descriptor_nltk_VBN,descriptor_nltk_VBP,descriptor_nltk_VBZ,descriptor_nltk_WDT,descriptor_nltk_WP,descriptor_nltk_WP$,descriptor_nltk_WRB
0,2511,Actual incident,Bolus was not prescribed by MD. Pt received 2 ...,Non prescribed bolus,NaN,NaN,None,Minor,No,TrueBeam 3,...,0,1,0,0,0,0,0,0,0,0
1,2521,Actual incident,Noticed by nutistionist (pt had app. with supp...,skin breakdown post RT,NaN,I handed over the investigation of incident to...,Mild,None,Unknown,STx 1,...,0,0,0,0,0,0,0,0,0,0
2,2562,Actual incident,"Ready for treatment task sent,completed,howeve...",Call patient task never sent,NaN,NaN,None,None,Unknown,STx 2,...,1,1,0,0,0,0,0,0,0,0
3,2560,Near miss,"Physics QA completed, but ready for treatment ...",ready for treatment tasking error,NaN,tasking error,NaN,NaN,NaN,Tasking,...,0,0,1,0,0,0,0,0,0,0
4,2526,Near miss,Typo in prescription document. It was prescrib...,prescription error,NaN,NaN,NaN,NaN,NaN,STx 2,...,0,0,0,0,0,0,0,0,0,0
5,2513,Actual incident,Started patient's treatments on a Friday inste...,New Start on Friday,NaN,NaN,None,None,Unknown,STx 1,...,0,0,0,0,0,0,0,0,0,0
6,2570,Actual incident,Pt was treated on TB 6on Feb 1st due to Tb3 be...,Lost vacloc,NaN,NaN,None,Minor,No,TrueBeam 6,...,0,0,0,1,0,0,0,0,0,0
7,2518,Near miss,Bolus not prescibed,Bolus,NaN,NaN,NaN,NaN,NaN,TrueBeam 3,...,0,0,0,0,0,0,0,0,0,0
8,2543,Actual incident,"Patient was lost(forgotten) in system, plan re...",Lack of adequate planning time.,NaN,patient lost in process,None,None,No,Treatment Planning,...,0,0,0,0,0,0,0,0,0,0
9,2541,Actual incident,Oncologist complained why patient was not cont...,Tasking Error,NaN,tasking error,None,None,No,Treatment Planning,...,0,0,1,0,0,0,0,0,0,0


In [402]:
analyzed.to_csv('analyzed_both.csv')